In [1]:
import pandas as pd
import pandas_ta as ta
import numpy as np
from scipy.signal import argrelextrema
from backtesting import Strategy, Backtest
from backtesting.lib import resample_apply

# Load data from CSV
data = pd.read_csv("1h_TATAPOWER.csv")
data['Datetime'] = pd.to_datetime(data['Datetime'])
data.set_index('Datetime', inplace=True)

# Calculate ATR
data['ATR'] = ta.atr(data.High, data.Low, data.Close, length=14)

# Find local maxima and minima
order = 10
max_idx = argrelextrema(data['High'].values, np.greater, order=order)[0]
min_idx = argrelextrema(data['Low'].values, np.less, order=order)[0]

# Create signal labels
labels = np.zeros(len(data))  # Initialize with non-pivot labels
labels[max_idx] = -1  # Set labels for pivot highs (sell signal)
labels[min_idx] = 1  # Set labels for pivot lows (buy signal)
data['Signal'] = labels


data = data.dropna()




data = data.dropna()


data = data[2000:]
    
# Define the strategy

def SMA(array, n):
    """Simple moving average"""
    return pd.Series(array).rolling(n).mean()

def RSI(array, n):
    """Relative strength index"""
    # Approximate; good enough
    gain = pd.Series(array).diff()
    loss = gain.copy()
    gain[gain < 0] = 0
    loss[loss > 0] = 0
    rs = gain.ewm(n).mean() / loss.abs().ewm(n).mean()
    return 100 - 100 / (1 + rs)



class System(Strategy):
    d_rsi = 30  # Daily RSI lookback periods
    w_rsi = 30  # Weekly
    level = 70
    
    def init(self):
        # Compute moving averages the strategy demands
        self.ma10 = self.I(SMA, self.data.Close, 10)
        self.ma20 = self.I(SMA, self.data.Close, 20)
        self.ma50 = self.I(SMA, self.data.Close, 50)
        self.ma100 = self.I(SMA, self.data.Close, 100)
        
        # Compute daily RSI(30)
        self.daily_rsi = self.I(RSI, self.data.Close, self.d_rsi)
        
        # To construct weekly RSI, we can use `resample_apply()`
        # helper function from the library
        self.weekly_rsi = resample_apply(
            'W-FRI', RSI, self.data.Close, self.w_rsi)
        
        
    def next(self):
        price = self.data.Close[-1]
        
        # If we don't already have a position, and
        # if all conditions are satisfied, enter long.
        if (not self.position and
            self.daily_rsi[-1] > self.level and
            self.weekly_rsi[-1] > self.level and
            self.weekly_rsi[-1] > self.daily_rsi[-1] and
            self.ma10[-1] > self.ma20[-1] > self.ma50[-1] > self.ma100[-1] and
            price > self.ma10[-1]):
            
            # Buy at market price on next open, but do
            # set 8% fixed stop loss.
            self.buy(sl=.92 * price)
        
        # If the price closes 2% or more below 10-day MA
        # close the position, if any.
        elif price < .98 * self.ma10[-1]:
            self.position.close()
            
bt = Backtest(data, System, cash=1000, margin=1/5, commission=0.05)
stats, heatmap = bt.optimize(
    d_rsi=range(10, 35, 5),
    w_rsi=range(10, 35, 5),
    level=range(30, 80, 10),
    # sl_coef=tuple(np.linspace(1, 2, 10)), 
    maximize='Return [%]',
    random_state=0,
    return_heatmap=True
        )

print(stats)
bt.plot()



E:\anaconda3\Lib\site-packages\backtesting\_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

  0%|          | 0/5 [00:00<?, ?it/s]

Start                     2024-03-04 14:15...
End                       2024-06-05 15:15...
Duration                     93 days 01:00:00
Exposure Time [%]                    1.658768
Equity Final [$]                    639.72525
Equity Peak [$]                        1000.0
Return [%]                         -36.027475
Buy & Hold Return [%]                 7.68838
Return (Ann.) [%]                  -84.204605
Volatility (Ann.) [%]               10.534045
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -36.227487
Avg. Drawdown [%]                  -36.227487
Max. Drawdown Duration       33 days 05:00:00
Avg. Drawdown Duration       33 days 05:00:00
# Trades                                    1
Win Rate [%]                              0.0
Best Trade [%]                      -7.482691
Worst Trade [%]                     -7.482691
Avg. Trade [%]                    

E:\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
E:\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Admin\AppData\Local\Temp\ipykernel_10324\1080925656.py:110: UserWarning: Superimposed OHLC plot matches the original plot. Skipping.
  bt.plot()
E:\anaconda3\Lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
E:\anaconda3\Lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p1415', ...)